### *Maciej Pawlikowski*
# AI4Games: projekt

Byłem członkiem grupy zajmującej się tworzeniem generatora map losowych do Heroes of Might and Magic III. Moja rola w projekcie polegała na napisaniu uproszczonej wersji silnika walki, a następnie wyznaczeniu współczynników odzwierciedlających wzajemną siłę potworków występujących w grze. Współczynniki te (prawdopodobnie) będą pomocne w 
podejmowaniu decyzji o rodzaju i liczbie stworów stawianych na planszy w procesie wypełniania mapy obiektami.

### Wartości jednostek

Sama gra posiada wewnętrzne wartości, które wykorzystuje do szacowania siły armii. Szczególnie interesującym atrybutem jest "AI Value", które na pierwszy rzut oka wygląda dosyć sensownie. Niestety każdy potwór ma tylko jedno AI Value. Z tego powodu nie można na podstawie tej wartości określić jak (w stosunku do innych monstrów) poradzi sobie w walce przeciwko pewnemu ograniczonemu podzbiorowi wszystkich jednostek, na przykład przeciwko jednostkom neutralnym, albo jednostkom konkretnego poziomu.

Wydaje się, że dobre oszacowanie wzajemnej siły kreatur można by uzyskać mając osobny współczynnik dla każdej *pary* jednostek. Pomogłoby to również wyznaczać względną jakość jednostek przy ograniczeniu do określonej grupy przeciwników. W tym projekcie przedstawiam propozycję takiego właśnie wartościowania.

### Estimation method

Effectiveness of creature $A$ in combat versus creature $B$ is calculated by simulating a large number of fights between those units. The aim is to find the stack sizes for $A$ and $B$ such that both stacks have more or less equal chances against each other. The exact method is as follows:

   - $A$, $B \leftarrow$ types of fighting creatures
   
   
   - $S_A$, $S_B \leftarrow$ stacks with some starting counts
   
   
   - simulate combat between $S_A$ and $S_B$, find $S_w$, $S_l$ - winning stack and losing stack (*)
   
   
   - $\mathrm{low} \leftarrow \mathrm{count}(S_l)$
   
   
   - while $S_l$ loses to $S_w$:
       * $\mathrm{low} \leftarrow \mathrm{count}(S_l)$
       * incease $\mathrm{count}(S_l)$ by some amount (I chose $10 \%$ of the starting stack size)
       
       
   - search in $(\mathrm{low}, \mathrm{count}(S_l))$ interval for a number $k$ of creatures in $S_l$, such that the result of $S_l$ vs $S_w$ is balanced
   
   
   - set $\mathrm{count}(S_l)$ to $k$
   
   
   - return $\dfrac{\mathrm{count}(S_A)}{\mathrm{count}(S_B)}$
   
   (*) If, for example, $S_l$ is $S_A$, then by changing $S_l$ later, we also change $S_A$.
   
Whenever I simulate combat between two stacks, I actually do $n$ fights (500 by default) and count wins of each side. A stack loses, if it won less than half of the fights. Combat result is "balanced" when the difference between numbers of wins is lower than 10% of $n$.

We also have to somehow set the starting sizes for both stacks. I used AI Value for that. In a fight between $A$ and $B$ I set $\mathrm{cout}(S_A)$ = $p * \mathrm{AI\_Value}(B)$ and $\mathrm{cout}(S_B)$ = $p * \mathrm{AI\_Value}(A)$, with $p=10$.

I didn't do mirror fights (same unit on both sides), the result is set to tie in those cases.

#### Example: Mighty Gorgon vs Cavalier
    
    n = 500
    
    Mighty Gorgon's AI Value = 1028
    Cavalier's      AI Value = 1946
    
    Starting counts: 19460, 10280
    
    Gorgons win 500 fights, Cavaliers' stack size up to 11308
    Gorgons win 500 fights, Cavaliers' stack size up to 12336
    Gorgons win 500 fights, Cavaliers' stack size up to 13364
    Gorgons win 500 fights, Cavaliers' stack size up to 14392
    Gorgons win 500 fights, Cavaliers' stack size up to 15420
    Gorgons win 500 fights, Cavaliers' stack size up to 16448
    Gorgons win 492 fights, Cavaliers' stack size up to 17476
    Gorgons win   2 fights, losing the combat

    Now binary search in (16448, 17476) for a balanced count.
    Middle is in 16962. We are lucky, because this already gives us balanced sizes:

        19460 Mighty Gorgons won 241 fights
        16962 Cavaliers      won 259 fights
            
    Result is 19460 / 16962 = 1.147

This estimation took 1.52s on a middle-class laptop.
    
We need approximately 1.147 Mighty Gorgons to kill a Cavalier. Additionaly we see that AI underestimates Mighty Gorgon a lot, because it doesn't know the value of its special ability.

### Combat simulation

*In this section, a *shooter* means a creature with ranged attack (i.e. has number of shots > 0), and a *walker* is a creature without one.*

The combat engine I use is greatly simplified, but in a vast majority of cases should be equivalent to the original one. I am only interested in battles between singular stacks (like 100 Archers vs 20 Crusaders), so there is no need to implement the entire battlefield. This allowed me to describe the flow of combat using very simple rules. There is no explicit stack movement or combat rounds. Defending does not exist; it can prolong the combat indefinitely and we can't have that. As far as waiting goes, there is no point to do it outside of the walker - shooter encounter described below.

I distinguish three basic cases:

    1) walker - walker
    2) shooter - shooter
    3) walker - shooter
    
**1) Walker - walker**

Simple case, the stacks keep attacking each other until one of them is dead. The stack with the higher speed starts, ties are broken at random.

**2) Shooter - shooter**

THe stacks keep shooting at each other until one of them is dead or one of them has no more ammunition. In the latter case we are reduced to the case (3). The order of attacks is the same as in case (1).

**3) Walker - shooter**

The most complicated case. In short: shooter shoots at walker until walker catches it, then they fight hand-to-hand. I calculate how many full-strength shots walker can avoid if it's smart.

Long version (Python-ish syntax):

    d  = number of hexes the walker has to travel to be able to attack
    ws = walker's speed
    
    if walker is slower than shooter (*):
        number_of_shots = (d // ws + (d % ws > 0))

        A walker can avoid at most one full shot. 
        To do so, it has to not enter the shooter's range when doing it's first move.
        Shooter is faster, so it can wait to force walker to move first, 
        which makes walker unable to avoid a full shot by waiting. 
        No creature in the game is slow enough to be outside of range after two rounds.
        
        Optimal length of the first move for a walker is m, calculated as follows:
            m = d % ws
            if m == 0:
                m = ws
        Every subsequent move has length ws. Making first move longer than m is
        unnecessary, making it any shorter increases the number of rounds without attack 
        for a walker.
        If shooter has no penalty after walker marches m hexes forward, than the walker
        can't profitably avoid any full shots. Otherwise, exactly one shot is fired with
        penalty. 
          
    if walker is faster than shooter (**):
        numer_of_shots = (d // ws - (d % ps == 0))

        In this case walker can avoid one additional shot, because it can wait to force
        shooter to shoot first. Up to one shot is avoided by smart movement, like in 
        the previous case. Overall, walker can avoid one or two full shots, depending on
        its speed.
      
    if walker and shooter have the same speed:
        In this case starting stack is chosen randomly. Waiting is not beneficial 
        anymore, because it changes the turn order for the rest of the fight. This makes
        walker unable to avoid any shots by waiting.
        
        number_of_shots is the same as in (*) if the shooter starts, and the same as in
        (**) otherwise.
        
    Now shooter shoots at walker number_of_shots times, some of which are penalized
    accordingly. After that we have case (1), with the walker being starting stack.

### Combat rules

I took into account every basic combat mechanic and creature characteristic, like attack, defense, number of shots, retaliation, creature size, etc. In addition to that, I implemented most special abilities:

  - no enemy retaliation
  - no meele penalty
  - double attack or shot
  - rebirth (Phoenix)
  - enemy defense reduction (Behemoth, Ancient Behemoth)
  - death blow (Dread Knight)
  - life drain (Vampire Lord)
  - death stare (Mighty Gorgon)
  - hate (e.g. Angel - Devil) and double damage (opposing Elementals)
  - fear (Azure Dragon)
  - regeneration (Wight, Wraith, Troll)
  - fire shield (Efreet Sultan)
  - acid breath (Rust Dragon)
  - lighning strike (Thunderbird)
  - aging (Ghost Dragon)
  - poison (Wyvern Monarch)
  - curse (Mummy, Black Knight, Dread Knight) (small discrepancy in duration - sometimes ends half a round too early)
  - weakness (Dragon Fly) (same as above)
  - disease (Zombie) (same as above)
  - blind (Unicorn, War Unicorn)
  - paralyzing venom (Scorpicore)
  - petrification (all Basilisks and Medusas)
  
In all cases (I hope) I consider the following creatures' immunities:

  - golems - less magic damage (%)
  - Firebird, Phoenix, Efreet, Efreet Sultan - immune to fire spells
  - all undeads - resistances of undeads
  - all elementals - resistances of elementals
  - gargoyles, golems - resistances of non-living creatures
  - Green Dragon, Red Dragon, Azure Dragon - immune to spells 1-3
  - Gold Dragon - immune to spells 1-4
  - Black Dragon, Magic Elemental - immune to all spells
  - Dwarf, Battle Dwarf, Crystal Dragon - magic resistance (%)
  - Giant, Titan - immune to mind spells
  - Troglodytes, Infernal Troglodytes - immune to blind and petrification
  
Not implemented:

  - any spells cast by creatures (most of them are useless in a duel anyway, others can take too much time to implement; this does not include incidental effects, like Efreet's Fire Shield or Dragon Fly's Weakness)
  - jousting (Cavalier, Champion)
  - strike and return (Harpy Hag)
  - abilities that are useless in a duel (like Dendroids' binding)
  - everything else I forgot

### Various stuff

Każda wartość zamieszczona w pliku *scores.csv* wyznacza iloraz liczebności oddziałów, przy którym wynik walki jest wyrównany. Jest tylko jeden problem: te wartości są stabilne dla dużych oddziałów. W przypadku walk kameralnych sprawa się komplikuje. Na przykład gdy pikinierzy zadadzą 700 obrażeń, to zabiją 2 Archanioły. Gdy zadzadzą 10 razy więcej, zabiją 28, czyli aż 14 razy więcej Archaniołów. Prowadzi to do tego, że 8 Archaniołów klepie bez problemu 510 Pikinierów, ale 80 Archaniołów i 5100 Pikinierów mają w miarę równe szanse. Wynika to z tego, że słabe uderzenia nie mają żadnego wpływu na zdolności bojowe jednostek (oddział atakuje słabiej dopiero, gdy przynajmniej jedna jednostka zginie). W związku z tym wyznaczone wartości będą faworyzowały jednostki wysokiego poziomu.

*-*

Obliczenie wszystkich wartości trwało około 5 godzin na nowoczesnym i7, bez żadnych optymalizacji, w Pythonie.

*-*

Mając dany pozdbiór stworków $A$ można, ze stratą informacji, wyznaczyć dla każdego potwora w grze jedną liczbę reprezentującą jego siłę w walce przeciwko $A$. Wystarczy wziąć sumę jego wspóczynników po wszystkich potworach w $A$. Po odpowiednim przeskalowaniu, tak obliczone wartości, dla $A$ będącego zbiorem wszystkich występujących w grze potworów, można bezpośrednio porównywać z AI Value. Mogą też one posłużyć jako parametry jakiegoś rozkładu prawdopodobieństwa, jeśli będziemy chcieli sensownie wylosować potworka, którego postawić na podwórku gracza.

*-*

Oprócz symulatora walk zrobiłem też prowizoryczny klikalny interfejs, w którym można przeglądać parametry potworków i wyniki walk oraz obliczać wartości opisane w poprzednim punkcie. Mam zamiar go jeszcze porządnie rozwinąć, ale na razie nie starczyło mi czasu.

*-*

Krótki Opis Ważnych Plików:

    combat_sim.ipynb - notebook do eksperymentów
    combat.py - kod walk 
    unit.py - reprezentacja jednostki i oddziału
    scores.csv - wyniki
    CRTRAITS.TXT - parametry potworków, zgodne z najnowszą angielską wersją
    
*-*

Przydatne linki:

http://h3.heroes.net.pl/uploaded/download/Heroes33patch.ZIP - nieoficjalny patch do polskiej wersji zawierający aktualne parametry stworów

http://heroes.thelazy.net/ - obszerne źródło informacji o H3

http://heroescommunity.com/viewthread.php3?TID=19321&pagenumber=2 - opis CRTRAITS.TXT

http://heroescommunity.com/viewthread.php3?TID=12210&pagenumber=2 - dyskusja o tym, jak zabijają krowy